In [13]:
%matplotlib inline
import os
import pandas as pd
from glob import glob
import numpy as np

In [14]:
from keras import layers
from keras import models
from keras.layers.advanced_activations import LeakyReLU
from keras.optimizers import Adam
import keras.backend as K
import librosa
import librosa.display
import pylab
import matplotlib.pyplot as plt
from matplotlib import figure
import gc
import pathlib

In [15]:
cmap = plt.get_cmap('inferno')

plt.figure(figsize=(10,10))
genres = 'blues classical country disco hiphop jazz metal pop reggae rock'.split()
genres = 'classical jazz metal pop rock'.split()
for g in genres:
    pathlib.Path(f'dataset/img_data/{g}').mkdir(parents=True, exist_ok=True)     
    for filename in os.listdir(f'./dataset/genres/{g}'):
        songname = f'./dataset/genres/{g}/{filename}'
        y, sr = librosa.load(songname, mono=True, duration=5)
        plt.specgram(y, NFFT=2048, Fs=2, Fc=0, noverlap=128, cmap=cmap, sides='default', mode='default', scale='dB');
        plt.axis('off');
        plt.savefig(f'dataset/img_data/{g}/{filename[:-3].replace(".", "")}.png')
        plt.clf()
 

<Figure size 720x720 with 0 Axes>

# Create a spectrogram

In [16]:
def create_spectrogram(filename,name):
    plt.interactive(False)
    clip, sample_rate = librosa.load(filename, sr=None)
    fig = plt.figure(figsize=[0.72,0.72])
    ax = fig.add_subplot(111)
    ax.axes.get_xaxis().set_visible(False)
    ax.axes.get_yaxis().set_visible(False)
    ax.set_frame_on(False)
    S = librosa.feature.melspectrogram(y=clip, sr=sample_rate)
    librosa.display.specshow(librosa.power_to_db(S, ref=np.max))
    filename  = 'working/train/' + name + '.jpg'
    plt.savefig(filename, dpi=400, bbox_inches='tight',pad_inches=0)
    plt.close()    
    fig.clf()
    plt.close(fig)
    plt.close('all')
    del filename,name,clip,sample_rate,fig,ax,S

In [17]:
def create_spectrogram_test(filename,name):
    plt.interactive(False)
    clip, sample_rate = librosa.load(filename, sr=None)
    fig = plt.figure(figsize=[0.72,0.72])
    ax = fig.add_subplot(111)
    ax.axes.get_xaxis().set_visible(False)
    ax.axes.get_yaxis().set_visible(False)
    ax.set_frame_on(False)
    S = librosa.feature.melspectrogram(y=clip, sr=sample_rate)
    librosa.display.specshow(librosa.power_to_db(S, ref=np.max))
    filename  = Path('working/test/' + name + '.jpg')
    fig.savefig(filename, dpi=400, bbox_inches='tight',pad_inches=0)
    plt.close()    
    fig.clf()
    plt.close(fig)
    plt.close('all')
    del filename,name,clip,sample_rate,fig,ax,S

# Extracting features

In [18]:
import csv

In [19]:
ids = []
labels = []

In [23]:
file = open('./dataset/data.csv', 'w', newline='')
header = ["id","class"]
with file:
    writer = csv.writer(file)
    writer.writerow(header)
for g in genres:
    for filename in os.listdir(f'./dataset/genres/{g}'):
        songname = f'./dataset/genres/{g}/{filename}'
        create_spectrogram(songname,filename)
        
        # extracts the  genre name
        labels.append(filename.split('.')[0])
        ids.append(filename + '.jpg')
        


In [24]:
df = pd.DataFrame({'ID': ids, 'Class': labels})

In [25]:
df.head()

,ID,Class
0,classical.00004.au.jpg,classical
1,classical.00001.au.jpg,classical
2,classical.00002.au.jpg,classical
3,classical.00003.au.jpg,classical
4,classical.00008.au.jpg,classical


# Creating a data generator

In [26]:
from keras_preprocessing.image import ImageDataGenerator
datagen = ImageDataGenerator(rescale=1/255, validation_split=.25)

In [27]:
train_generator = datagen.flow_from_dataframe(
    dataframe = df,
    directory="working/train/",
    x_col = "ID",
    y_col = "Class",
    subset="training",
    batch_size=32,
    seed=42,
    shuffle=True,
    class_mode="categorical",
    target_size=(64, 64)
)

Found 38 images belonging to 5 classes.


In [28]:
from keras.layers import Dense, Activation, Flatten, Dropout, BatchNormalization
from keras.models import Sequential, Model
from keras.layers import Conv2D, MaxPooling2D
from keras import regularizers, optimizers
import pandas as pd
import numpy as np

model = Sequential()
model.add(Conv2D(32, (3, 3), padding='same',
                 input_shape=(64,64,3)))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.5))
model.add(Conv2D(128, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(128, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.5))
model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(5, activation='softmax'))
model.compile(optimizers.rmsprop(lr=0.0005, decay=1e-6),loss="categorical_crossentropy",metrics=["accuracy"])
model.summary()

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 64, 64, 32)        896       
_________________________________________________________________
activation_1 (Activation)    (None, 64, 64, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 62, 62, 64)        18496     
_________________________________________________________________
activation_2 (Activation)    (None, 62, 62, 64)        0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 31, 31, 64)        0         
_________________________________________________________________
dropout_1 (Dropout)  

In [29]:
#Fitting keras model, no test gen for now
STEP_SIZE_TRAIN=train_generator.n//train_generator.batch_size
#STEP_SIZE_TEST=test_generator.n//test_generator.batch_size
model.fit_generator(generator=train_generator,
                    steps_per_epoch=STEP_SIZE_TRAIN,
                    epochs=150
)


Instructions for updating:
Use tf.cast instead.
Epoch 1/150
1/1 [==============================] - 1s 1s/step - loss: 1.6071 - acc: 0.3125
Epoch 2/150
1/1 [==============================] - 0s 214ms/step - loss: 4.5988 - acc: 0.1667
Epoch 3/150
1/1 [==============================] - 1s 892ms/step - loss: 1.5730 - acc: 0.2812
Epoch 4/150
1/1 [==============================] - 0s 194ms/step - loss: 1.5907 - acc: 0.1667
Epoch 5/150
1/1 [==============================] - 1s 876ms/step - loss: 1.4935 - acc: 0.2812
Epoch 6/150
1/1 [==============================] - 0s 193ms/step - loss: 1.3805 - acc: 0.3333
Epoch 7/150
1/1 [==============================] - 1s 888ms/step - loss: 1.4234 - acc: 0.2500
Epoch 8/150
1/1 [==============================] - 0s 188ms/step - loss: 1.6450 - acc: 0.0000e+00
Epoch 9/150
1/1 [==============================] - 1s 874ms/step - loss: 1.4621 - acc: 0.2188
Epoch 10/150
1/1 [==============================] - 0s 196ms/step - loss: 1.5757 - acc: 0.1667
Epoch 11/1

Epoch 85/150
1/1 [==============================] - 1s 914ms/step - loss: 0.6977 - acc: 0.6562
Epoch 86/150
1/1 [==============================] - 0s 191ms/step - loss: 0.1667 - acc: 1.0000
Epoch 87/150
1/1 [==============================] - 1s 878ms/step - loss: 0.5164 - acc: 0.7500
Epoch 88/150
1/1 [==============================] - 0s 187ms/step - loss: 0.8197 - acc: 0.8333
Epoch 89/150
1/1 [==============================] - 1s 905ms/step - loss: 1.3388 - acc: 0.5938
Epoch 90/150
1/1 [==============================] - 0s 211ms/step - loss: 0.8299 - acc: 0.5000
Epoch 91/150
1/1 [==============================] - 1s 1s/step - loss: 0.6110 - acc: 0.7500
Epoch 92/150
1/1 [==============================] - 0s 363ms/step - loss: 0.4446 - acc: 0.8333
Epoch 93/150
1/1 [==============================] - 2s 2s/step - loss: 0.5886 - acc: 0.7500
Epoch 94/150
1/1 [==============================] - 0s 317ms/step - loss: 0.5924 - acc: 0.8333
Epoch 95/150
1/1 [==============================] - 1s 1

# Transfer Learning

In [35]:
from keras.preprocessing.image import load_img, img_to_array

In [32]:
SAMPLE_SONG_PATH = "working/train/classical.00000.au.jpg"
STYLE_SONG_PATH = "working/train/jazz.00000.au.jpg"

In [33]:
TARGET_HEIGHT = 217
TARGET_WIDTH = 223
TARGET_SIZE = (TARGET_HEIGHT, TARGET_WIDTH)

In [36]:
cImage = load_img(path=SAMPLE_SONG_PATH, target_size=TARGET_SIZE)
cImArr = img_to_array(cImage)
cImArr = K.variable(cImArr)

In [ ]:

sImage = load_img(path=SAMPLE_SONG_PATH, target_size=TARGET_SIZE)
sImArr = img_to_array(sImage)
sImArr = K.variable(cImArr)